In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('TRAIN_Reco_2021_2022_2023.parquet.gzip').reset_index()
df.head()

,ExecutionTime,ID,high,low,close,volume
0,2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
1,2021-01-06 22:00:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2,2021-01-06 22:15:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
3,2021-01-06 22:30:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
4,2021-01-06 22:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0


In [3]:
df.shape

(69805344, 5)

In [4]:
df.columns

Index(['ID', 'high', 'low', 'close', 'volume'], dtype='object')

In [2]:
# Handle duplicate 'ExecutionTime' by adding a small offset to create unique timestamps
df['ExecutionTime'] += pd.to_timedelta(df.groupby('ExecutionTime').cumcount(), unit='ms')

# Set 'ExecutionTime' as index
df.set_index('ExecutionTime', inplace=True)

# Resample to 15-minute intervals
resampled_df = df.resample('15T').asfreq()

# Interpolating numeric columns
resampled_df[['high', 'low', 'close', 'volume']] = resampled_df[['high', 'low', 'close', 'volume']].interpolate()

# Forward fill the ID column (or use another method based on context)
resampled_df['ID'] = resampled_df['ID'].fillna(method='ffill')

# Reset index to make it more usable
resampled_df.reset_index(inplace=True)

# Display the result
print(resampled_df)

C:\Users\a4293604\AppData\Local\Temp\ipykernel_26952\577089354.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.resample('15T').asfreq()


                   ExecutionTime       ID  high  low  close  volume
0      2021-01-06 21:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
1      2021-01-06 22:00:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
2      2021-01-06 22:15:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
3      2021-01-06 22:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
4      2021-01-06 22:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
...                          ...      ...   ...  ...    ...     ...
103872 2023-12-24 21:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
103873 2023-12-24 22:00:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
103874 2023-12-24 22:15:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
103875 2023-12-24 22:30:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
103876 2023-12-24 22:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0

[103877 rows x 6 columns]


C:\Users\a4293604\AppData\Local\Temp\ipykernel_26952\577089354.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  resampled_df['ID'] = resampled_df['ID'].fillna(method='ffill')


In [5]:
# Create a full range of time, from the min to the max ExecutionTime, with 15-minute intervals
full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')

# Reindex the dataframe to include all the 15-minute intervals without dropping any original data
df_full = df.reindex(full_time_index.union(df.index))

# Interpolating numeric columns for the newly added 15-minute intervals
df_full[['high', 'low', 'close', 'volume']] = df_full[['high', 'low', 'close', 'volume']].interpolate()

# Forward fill the ID column (or use another method based on context)
df_full['ID'] = df_full['ID'].fillna(method='ffill')

# Reset index to make it more usable
df_full.reset_index(inplace=True)

# Display the result
print(df_full)

C:\Users\a4293604\AppData\Local\Temp\ipykernel_26952\1975498970.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='15T')
C:\Users\a4293604\AppData\Local\Temp\ipykernel_26952\1975498970.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_full['ID'] = df_full['ID'].fillna(method='ffill')


                                    index       ID  high  low  close  volume
0               2021-01-06 21:45:00+01:00  Fri00Q1   0.0  0.0    0.0     0.0
1        2021-01-06 21:45:00.001000+01:00  Fri00Q2   0.0  0.0    0.0     0.0
2        2021-01-06 21:45:00.002000+01:00  Fri00Q3   0.0  0.0    0.0     0.0
3        2021-01-06 21:45:00.003000+01:00  Fri00Q4   0.0  0.0    0.0     0.0
4        2021-01-06 21:45:00.004000+01:00  Fri01Q1   0.0  0.0    0.0     0.0
...                                   ...      ...   ...  ...    ...     ...
69805339 2023-12-24 22:45:00.667000+01:00  Wed22Q4   0.0  0.0    0.0     0.0
69805340 2023-12-24 22:45:00.668000+01:00  Wed23Q1   0.0  0.0    0.0     0.0
69805341 2023-12-24 22:45:00.669000+01:00  Wed23Q2   0.0  0.0    0.0     0.0
69805342 2023-12-24 22:45:00.670000+01:00  Wed23Q3   0.0  0.0    0.0     0.0
69805343 2023-12-24 22:45:00.671000+01:00  Wed23Q4   0.0  0.0    0.0     0.0

[69805344 rows x 6 columns]


In [14]:
df.reset_index()


,ExecutionTime,ID,high,low,close,volume
0,2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
1,2021-01-06 22:00:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
2,2021-01-06 22:15:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
3,2021-01-06 22:30:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
4,2021-01-06 22:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
69805339,2023-12-24 21:45:00.671000+01:00,Wed23Q4,0.0,0.0,0.0,0.0
69805340,2023-12-24 22:00:00.671000+01:00,Wed23Q4,0.0,0.0,0.0,0.0
69805341,2023-12-24 22:15:00.671000+01:00,Wed23Q4,0.0,0.0,0.0,0.0
69805342,2023-12-24 22:30:00.671000+01:00,Wed23Q4,0.0,0.0,0.0,0.0


In [17]:
df_full

,index,ID,high,low,close,volume
0,2021-01-06 21:45:00+01:00,Fri00Q1,0.0,0.0,0.0,0.0
1,2021-01-06 21:45:00.001000+01:00,Fri00Q2,0.0,0.0,0.0,0.0
2,2021-01-06 21:45:00.002000+01:00,Fri00Q3,0.0,0.0,0.0,0.0
3,2021-01-06 21:45:00.003000+01:00,Fri00Q4,0.0,0.0,0.0,0.0
4,2021-01-06 21:45:00.004000+01:00,Fri01Q1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
69805339,2023-12-24 22:45:00.667000+01:00,Wed22Q4,0.0,0.0,0.0,0.0
69805340,2023-12-24 22:45:00.668000+01:00,Wed23Q1,0.0,0.0,0.0,0.0
69805341,2023-12-24 22:45:00.669000+01:00,Wed23Q2,0.0,0.0,0.0,0.0
69805342,2023-12-24 22:45:00.670000+01:00,Wed23Q3,0.0,0.0,0.0,0.0


In [18]:
import pandas as pd

# Assuming `df` is the original DataFrame and `df_full` is the reindexed/resampled DataFrame

# Step 1: Identify the newly added rows by finding the indices that are in `df_full` but not in `df`
new_rows = df_full.index.difference(df.index)

# Step 2: Extract the newly added rows from the resampled DataFrame
added_rows = df_full.loc[new_rows]

# Step 3: Compare the common rows between the original DataFrame and the resampled DataFrame
# Reindex the original DataFrame to align with `df_full`
aligned_df = df.reindex(df_full.index)

# Find the differences between the aligned original DataFrame and the resampled DataFrame
differences = aligned_df != df_full

# Identify rows that have differences (excluding NaN rows from the original data that were interpolated)
rows_with_differences = differences.any(axis=1) & df_full.index.isin(df.index)

# Extract the rows that are different
modified_rows = df_full[rows_with_differences]

# Display the results
print(f"Number of added rows (new intervals): {len(added_rows)}")
print("Added rows (new intervals):")
print(added_rows)

print(f"\nNumber of modified rows (common rows with different values): {len(modified_rows)}")
print("Modified rows (common rows with different values):")
print(modified_rows)


ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects